# Story Planner Exercise

In this exercise, we'll create a story planning system that:
1. Breaks down story ideas into plot points
2. Develops characters and settings
3. Stores story elements in memory

Focus on experimenting with creative writing prompts!

In [ ]:
import os
import sys

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)

sys.path.append(grandparent_dir)

### 1. Setup 

In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory
from semantic_kernel.memory.volatile_memory_store import VolatileMemoryStore
from semantic_kernel.planners import SequentialPlanner

kernel = Kernel()

# Chat service
service_id = "chat-gpt"
chat_service = AzureChatCompletion(
    service_id=service_id,
)
kernel.add_service(chat_service)

# Embedding service
embedding_service_id = "embeddings"
kernel.add_service(AzureTextEmbedding(service_id=embedding_service_id))

# Memory setup
memory_store = VolatileMemoryStore()
memory = SemanticTextMemory(storage=memory_store, embeddings_generator=kernel.get_service(embedding_service_id))

# Planner setup
planner = SequentialPlanner(kernel,service_id)

### 2. Define Base Prompts (Experiment with these!)

These prompts work together to plan stories. Try modifying them to change how stories are developed!

In [ ]:
# Prompt 1: Create story outline
# Try changing the structure, adding elements, or modifying the style!
outline_prompt = """
Story Concept: {{$concept}}
Genre: {{$genre}}
Target Length: {{$length}}

Create a story outline that:
1. Follows a clear narrative arc
2. Includes compelling plot points
3. Builds tension effectively
4. Delivers a satisfying resolution

Format the outline as:
📖 Title Suggestion: [creative title]

🌟 Hook: [opening hook]

📈 Plot Points:
1. [setup/introduction]
2. [rising action]
3. [complication]
4. [climax]
5. [resolution]

🎯 Theme: [main theme]
"""

# Prompt 2: Develop characters
# Try adding different traits or changing the personality aspects!
character_prompt = """
Story Context: {{$story_outline}}
Character Role: {{$role}}

Create a detailed character profile that fits this story.
Make them complex and interesting, with clear motivations.

Format the profile as:
👤 Name: [full name]

🎭 Personality:
- Traits: [key personality traits]
- Motivations: [what drives them]
- Fears: [what they fear]

📜 Background:
- History: [relevant background]
- Skills: [abilities/expertise]
- Relationships: [key connections]

🔄 Character Arc:
- Starting Point: [initial state]
- Development: [how they change]
- End State: [final state]
"""

# Prompt 3: Design setting
# Try adding atmosphere, history, or cultural elements!
setting_prompt = """
Story Context: {{$story_outline}}
Location Type: {{$location}}

Create a rich and detailed setting that enhances the story.
Make it feel alive and integral to the plot.

Format the description as:
🌍 Name: [location name]

📍 Physical Description:
- Layout: [physical details]
- Atmosphere: [sensory details]
- Notable Features: [unique elements]

🕰️ Background:
- History: [relevant past]
- Culture: [social elements]
- Current State: [present situation]

🎬 Story Relevance:
- Plot Impact: [how it affects the story]
- Character Impact: [how it affects characters]
- Mood: [emotional atmosphere]
"""

# Create semantic functions
create_outline = kernel.add_function(
    prompt=outline_prompt,
    function_name="CreateOutline",
    plugin_name="StoryPlanner",
)

develop_character = kernel.add_function(
    prompt=character_prompt,
    function_name="DevelopCharacter",
    plugin_name="StoryPlanner",
)

design_setting = kernel.add_function(
    prompt=setting_prompt,
    function_name="DesignSetting",
    plugin_name="StoryPlanner",
)

### 3. Example Usage

Try these examples, then modify the prompts above to see how it changes the results!

In [ ]:
# Example 1: Create a story outline
story_details = {
    "concept": "A chef discovers they can taste the emotions of anyone who eats their food",
    "genre": "magical realism",
    "length": "short story"
}

outline = await kernel.invoke(
    create_outline,
    **story_details
)
print("Story Outline:\n", outline)

# Example 2: Develop main character
character = await kernel.invoke(
    develop_character,
    story_outline=str(outline),
    role="protagonist"
)
print("\nMain Character:\n", character)

# Example 3: Design primary setting
setting = await kernel.invoke(
    design_setting,
    story_outline=str(outline),
    location="restaurant"
)
print("\nPrimary Setting:\n", setting)

# Store elements in memory
await memory.save_information(
    collection="story_elements",
    id="outline_1",
    text=str(outline)
)

await memory.save_information(
    collection="story_elements",
    id="character_1",
    text=str(character)
)

await memory.save_information(
    collection="story_elements",
    id="setting_1",
    text=str(setting)
)

### 4. Search Story Elements

Try searching for elements you've stored!

In [ ]:
# Search for story elements
search_terms = ["character motivation", "plot twist", "emotional atmosphere"]

for term in search_terms:
    print(f"\nSearching elements related to '{term}':\n")
    results = await memory.search("story_elements", term)
    for result in results:
        print(f"Match (relevance: {result.relevance}):\n{result.text}\n")

### 5. Additional exercises

Try these exercises to experiment with story development prompts:

1. Modify the outline_prompt to:
   - Add subplots
   - Include plot twists
   - Develop parallel storylines

2. Enhance the character_prompt with:
   - Physical descriptions
   - Speech patterns
   - Personal quirks

3. Expand the setting_prompt to include:
   - Weather patterns
   - Time period details
   - Social dynamics

4. Create a new prompt that:
   - Generates dialogue between characters
   - Creates scene descriptions
   - Develops symbolic elements

5. Advanced Challenge:
   - Create a prompt that weaves multiple storylines together
   - Design character relationship dynamics
   - Build world mythology and history

Remember: Focus on how different prompt elements affect the story development. Try various styles and approaches!